# Importar os módulos necessários

In [106]:
import warnings
warnings.filterwarnings("ignore")

In [107]:
import pandas as pd
#necessario por causa do metodo de avaliaçao
from sklearn.model_selection import KFold
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

# Definir as Métricas para Avaliação dos Modelos

In [108]:
#metricas regressao
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def custom_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, r2


In [109]:
#metricas classificação
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [110]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
import numpy as np

# Função para calcular o RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Dicionário de métricas de regressão
REGRESSION_METRICS = {
    "MSE": make_scorer(mean_squared_error),
    "MAE": make_scorer(mean_absolute_error),
    "R2": make_scorer(r2_score),
    "RMSE": make_scorer(root_mean_squared_error)  # Adicionando o RMSE
}


In [111]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

# Ler o Conjunto de Dados

In [112]:
d = pd.read_csv("Acenario2Ideia2Mapeamento2.csv")
d

,id_transmissao,id_body_type,id_modelos,id_categoria_marca,mileage,ano,id_city,id_insurance,seats,engine_capacity,id_fuel_type,kms_driven,max_power,id_owner_type,resale_price_Lakh
0,0.14299,0.374291,3.101685,0.282021,23.84000,2019.0,1.091153,0.260439,5.0,1199.0,0.187053,30910.0,83.8,0.153049,5.66
1,0.14299,0.597152,2.418801,0.282021,17.50000,2018.0,1.289379,1.319603,5.0,1199.0,0.187053,48089.0,88.7,1.347893,6.64
2,0.14299,0.561469,2.599009,0.282021,17.40000,2015.0,0.916735,0.260439,5.0,1497.0,0.187053,51000.0,117.3,0.616480,5.65
3,0.55179,0.597152,3.665956,0.282021,14.08000,2021.0,0.916735,0.260439,7.0,1956.0,0.500118,30000.0,167.6,0.153049,23.00
4,0.55179,0.374291,2.762866,0.527127,21.40000,2019.0,1.172266,1.319603,5.0,1197.0,0.187053,61113.0,83.1,0.153049,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,0.55179,1.362760,3.665956,0.997260,19.49405,2021.0,1.033499,0.260439,7.0,2199.0,0.500118,80000.0,197.2,0.616480,26.50
13898,0.14299,0.374291,2.966986,0.282021,19.49405,2017.0,1.220871,0.260439,5.0,1197.0,0.187053,17923.0,81.8,0.153049,5.87
13899,0.14299,0.561469,3.297979,0.997260,19.49405,2017.0,1.033499,0.438155,5.0,1498.0,0.500118,63389.0,108.6,0.616480,7.43
13900,0.14299,0.597152,2.762866,0.527127,19.49405,2017.0,0.869613,0.438155,5.0,1248.0,0.500118,40000.0,88.5,0.616480,9.45


In [113]:
X, y = d.drop("resale_price_Lakh", axis=1), d["resale_price_Lakh"]

# Definir o Método de Validação Cruzada

In [114]:
#Classificação
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)


# Implementação dos Algoritmos de Machine Learning

### Árvores de Decisão

In [115]:
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Definir o regressor e o método de validação cruzada
dt_regressor = DecisionTreeRegressor(max_depth=12, random_state=1234)
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = {k: v.mean() for k, v in scores.items()}  # Negar os valores para tornar os scores positivos
pd.DataFrame(dt_scores, index=[0])



,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.084223,0.003164,14.632131,1.485408,0.882628,3.727914


In [116]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continue com a validação cruzada e a avaliação de métricas
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=1234)
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T





,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.073355,0.003007,20.452286,1.57011,0.831056,4.31727


In [117]:
dt = DecisionTreeRegressor(max_depth=3, random_state=1234)
splitter = KFold(10, random_state=1234, shuffle=True)
scores = cross_validate(dt, X, y, cv=splitter, scoring=REGRESSION_METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T

,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.02511,0.003348,28.587618,2.921302,0.763341,5.324555


In [118]:


# Criar um modelo de regressão de árvore de decisão
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=1234)

# Estratégia de validação cruzada (KFold) para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continuar com a validação cruzada e avaliação de métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T



,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,0.026423,0.011111,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Redes Neuronais

In [119]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=20, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,2.293647,0.004707,68.021469,4.709325,0.438593,8.153533


In [120]:
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Gerar dados de exemplo
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

# Criar um regressor de floresta aleatória
rf_regressor = RandomForestRegressor(max_depth=10, random_state=0)



# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=5, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
print(pd.DataFrame(rf_scores.mean()).T)
 


   fit_time  score_time   test_MSE  test_MAE   test_R2  test_RMSE
0  0.136313    0.004778  96.022625  7.399889  0.933136   9.583758


### Naive Bayes

### Support Vector Machine